In [3]:
import numpy as np
import mne
import scipy.signal as signal
import scipy.io.wavfile as wav
from mne_bids import BIDSPath, write_raw_bids
from pyxdf import resolve_streams, match_streaminfos
from mnelab.io.xdf import read_raw_xdf

In [6]:
audio_file = 'BIDS/sub-01/ses-01/audio/sub-01_ses-01_task-PilotStudy_run-01_audio.wav'
auido_events = 'BIDS/sub-01/ses-01/audio/sub-01_ses-01_task-PilotStudy_run-01_events.tsv'
eeg_file = 'BIDS/sub-01/ses-01/eeg/sub-01_ses-01_task-PilotStudy_run-01_eeg.edf'

In [8]:
eeg = mne.io.read_raw_edf(eeg_file, preload=True)

Extracting EDF parameters from /home/owaismujtaba/projects/PilotStudy/BIDS/sub-01/ses-01/eeg/sub-01_ses-01_task-PilotStudy_run-01_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 6306999  =      0.000 ...  6306.999 secs...


In [12]:
eeg.annotations

<Annotations | 9400 segments: RealSyllablesExperimentEndFixation:Audio_BA ...>

In [31]:
def get_eeg_events(annotations):
    onsets = []
    descriptions = []
    for item in annotations:
        stimului = item['description']
        if 'Speech' not in stimului or 'Words' not in stimului or 'Real' not in stimului:
            continue
        if 'Experiment' not in stimului:
            continue
        onsets.append(item['onset'])
        descriptions.append(item['description'])

    return np.array(onsets), descriptions
    

In [36]:
def convert_seconds_to_minutes_and_seconds(total_seconds):
    """
    Converts a time value in seconds into minutes and seconds.

    Args:
        total_seconds (float): The time in seconds.

    Returns:
        tuple: (minutes, seconds), where seconds includes fractional seconds.
    """
    minutes = int(total_seconds // 60)  # Integer division for minutes
    seconds = total_seconds % 60       # Remaining seconds (can include fractions)
    return f'{minutes}.{seconds}'

In [38]:
onsets_eeg, descriptions_eeg = get_eeg_events(eeg.annotations)

In [39]:
for index in range(s.shape[0]):
    print(convert_seconds_to_minutes_and_seconds(onsets_eeg[index]), descriptions_eeg[index])

42.49.45334099999991 RealWordsExperimentStartSpeech:Pictures_poncho
42.51.453133999999864 RealWordsExperimentEndSpeech:Pictures_poncho
42.55.586549000000105 RealWordsExperimentStartSpeech:Text_bufanda
42.57.58665400000018 RealWordsExperimentEndSpeech:Text_bufanda
43.1.7200410000000375 RealWordsExperimentStartSpeech:Text_clarinete
43.3.7200219999999717 RealWordsExperimentEndSpeech:Text_clarinete
43.7.6200770000000375 RealWordsExperimentStartSpeech:Audio_silence
43.9.6199459999998 RealWordsExperimentEndSpeech:Audio_silence
43.14.020089999999982 RealWordsExperimentStartSpeech:Audio_campana
43.16.019874000000073 RealWordsExperimentEndSpeech:Audio_campana
43.19.87009900000021 RealWordsExperimentStartSpeech:Pictures_zapato
43.21.86994700000014 RealWordsExperimentEndSpeech:Pictures_zapato
43.25.986825999999837 RealWordsExperimentStartSpeech:Text_bolso
43.27.986601999999948 RealWordsExperimentEndSpeech:Text_bolso
43.31.636920999999802 RealWordsExperimentStartSpeech:Text_guidía
43.33.6367599999

In [45]:
def get_audio_events(events):
    onsets = []
    descriptions = []
    
    for index in range(events.shape[0]):
        stimului = events['description'][index]
        if 'Speech' not in stimului or 'Words' not in stimului or 'Real' not in stimului:
            continue
        if 'Experiment' not in stimului:
            continue
        onsets.append(events['onset'][index])
        descriptions.append(events['description'][index])

    return np.array(onsets), descriptions

In [44]:
import pandas as pd
events = pd.read_csv(auido_events, delimiter='\t')
events['description'][0]

'SilentSyllablesPracticeStartFixation:Text_FAS'

In [46]:
o, d = get_audio_events(events)

In [51]:
lengths = []
for index in range(0, o.shape[0],2):
    print(convert_seconds_to_minutes_and_seconds(onsets_eeg[index]), descriptions_eeg[index])
    lengths.append(o[index+1]- o[index])

42.49.45334099999991 RealWordsExperimentStartSpeech:Pictures_poncho
42.55.586549000000105 RealWordsExperimentStartSpeech:Text_bufanda
43.1.7200410000000375 RealWordsExperimentStartSpeech:Text_clarinete
43.7.6200770000000375 RealWordsExperimentStartSpeech:Audio_silence
43.14.020089999999982 RealWordsExperimentStartSpeech:Audio_campana
43.19.87009900000021 RealWordsExperimentStartSpeech:Pictures_zapato
43.25.986825999999837 RealWordsExperimentStartSpeech:Text_bolso
43.31.636920999999802 RealWordsExperimentStartSpeech:Text_guidía
43.37.95350700000017 RealWordsExperimentStartSpeech:Audio_guidía
43.43.68674400000009 RealWordsExperimentStartSpeech:Pictures_micrófono
43.49.986902999999984 RealWordsExperimentStartSpeech:Pictures_vaso
43.55.97008400000004 RealWordsExperimentStartSpeech:Text_gato
44.2.236765000000105 RealWordsExperimentStartSpeech:Text_mono
44.8.553629999999885 RealWordsExperimentStartSpeech:Pictures_gato
44.14.336823999999979 RealWordsExperimentStartSpeech:Audio_espátula
44.20.

In [53]:
min(lengths), max(lengths)

(1.9996275991998118, 2.000218915016376)